**Pré-Tutorial**

In [ ]:
!pip install yfinance
!pip install --upgrade matplotlib
#!pip install matplotlib==3.1.3

In [1]:
#Calcular RSI
####### Compute RSI
#1º) Calcular Variação
#2º) Identificar se é um ganho ou perda
#3º) Calcular media movel (suavizada) de ganho e perda (janela definida pelo user)
#4º) Calcular RS
#5º) Calcular RSI

def indicador_RSI(df,window):
  df['change'] = df['Adj Close'] - df['Adj Close'].shift(1) #Calcular diferença entre fecho 

  df['gain'] = df.loc[df['change']>0,'change'].apply(abs) #Calcular Ganhos
  df.loc[(df['gain'].isna()),'gain']= 0 
  data[0,data.columns.get_loc('gain')] = np.NaN #df.loc[0,'gain'] = np.NaN


  df['loss'] = df.loc[df['change']<0,'change'].apply(abs) #Calcular Perdas
  df.loc[(df['loss'].isna()),'loss']= 0
  data[0,data.columns.get_loc('loss')] = np.NaN #df.loc[0,'loss'] = np.NaN

  ########### Average Gain/Loss
  #Normalmente são considerados 14 periodos (neste caso vou usar dias)
  df['avg_gain'] = df['gain'].rolling(window).mean()
  df['avg_loss'] = df['loss'].rolling(window).mean()

  #não pode ser vetorizado porque é uma formula recursiva
  first = df['avg_gain'].first_valid_index() #identificar primeiro valor que não seja nulo/na

  for index, row in df.iterrows():
    if index == first: #gravar o primeiro não nulo
      prev_avg_gain = row['avg_gain']
      prev_avg_loss = row['avg_loss']
    elif index > first: #calcular os seguintes (após o não nulo)
      df.loc[index,'avg_gain']= ((prev_avg_gain*(window -1)) + row['gain'])/window
      prev_avg_gain = df.loc[index,'avg_gain']

      df.loc[index,'avg_loss']= ((prev_avg_loss*(window -1)) + row['loss'])/window
      prev_avg_loss = df.loc[index,'avg_loss']

  #### Compute RS e RSI
  df[f'RS{window}'] = df['avg_gain']/df['avg_loss']
  df[f'RSI{window}'] = 100 - (100 / (1 + df[f'RS{window}']))

  #Filter Columns
  lista = df.columns.to_list()
  matching = [s for s in lista if "RSI" in s]
  sel_col = ['Adj Close'] + matching

  return df[sel_col].copy()

**Tutorial**

In [10]:
import matplotlib.pyplot as plt
import numpy as np
import yfinance as yf
import math

In [ ]:
data = yf.download('TSLA','2016-01-01','2020-10-01')
data.head()

In [ ]:
data_frame = indicador_RSI(data,14)
data_frame.head(20)

In [13]:
filter_sell = (data_frame['RSI14'].shift(1)<70) & (data_frame['RSI14']>70)
data_frame.loc[filter_sell,'Sell_Signal'] = data_frame.loc[filter_sell,'Adj Close']

In [14]:
filter_buy = (data_frame['RSI14'].shift(1)>30) & (data_frame['RSI14']<30)
data_frame.loc[filter_buy,'Buy_Signal'] = data_frame.loc[filter_buy,'Adj Close']

In [15]:
data_frame = data_frame.reset_index()
filtered_data = data_frame.loc[(data_frame['Date']<'2020-12-01')&(data_frame['Date']>'2018-12-01')]

In [16]:
plt.rcParams['figure.figsize'] = [6,4]
plt.rcParams['figure.dpi'] = 100

In [ ]:
fig, ax = plt.subplots()

#Plot 1
ax.plot(filtered_data['Date'],filtered_data['Adj Close'],label = 'Adj Close')

#Plot 2
ax2 = ax.twinx()
ax2.plot(filtered_data['Date'],filtered_data['RSI14'],label='RSI14',color='gray',alpha=0.5)
ax2.axhline(70,color='gray',ls='--',alpha=0.5)
ax2.axhline(30,color='gray',ls='--',alpha=0.5)
ax2.set_ylabel('RSI14')
ax2.legend()

ax.scatter(filtered_data['Date'],filtered_data['Buy_Signal'],label='Buy',marker='^',color='green',s=100)
ax.scatter(filtered_data['Date'],filtered_data['Sell_Signal'],label='Sell',marker='v',color='red',s=100)

ax.yaxis.set_major_formatter('${x:}')
fig.autofmt_xdate()

ax.set_title('Estratégia RSI')
ax.set_ylabel('Adj_Close')
ax.set_xlabel('Data')
ax.legend()